# Importation de la fonction pipeline

In [29]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_path)

from Application.data_processing import pipeline

# Définition des paramètres

Il faut défnir le chemin où sont stockés les données. <br>
Quel est la variable cible. <br>
A partir de quelle variable on veut créer nos subsets. <br>
Comment on nomme l'expérience dans mlflow. <br>

In [30]:
chemin = '../Data/datanba/'
cible ='Rounds'
critère = 'Pace'
experience_name ='Classification_Playoffs'

# Lancement du pipeline

On peut uniquement lancer la fonction pipeline qui s'occupe de tout.

In [ ]:
resultat = pipeline(chemin,cible,critère,experience_name)
print(resultat)

# Tout le processus

Ou bien c'est possible, si l'on veut passer au travers de certaines étapes, de reprendre chacunes des fonctions et executer celle que l'on souhaite.

# Importation des packages

In [52]:
from Infrastucture.data_loading import load_data
from Infrastucture.data_cleaning import clean_data, subsets
from Domain.data_preprocessing import preprocessing
from Domain.data_modelisation import ModelTrainer, model_testing
from Application.data_processing import sauvegarder_modele, charger_modele, pipeline
from sklearn.model_selection import train_test_split

# Définition des paramètres

In [33]:
chemin = '../Data/datanba/'
cible ='Rounds'
critère = 'Pace'
experience_name ='Classification_Playoffs'

# Chargement des données

In [34]:
df = load_data(chemin)

# Traitement des données

In [35]:
df = clean_data(df)

Attend./G: 11 valeurs nulles
AS: 29 valeurs nulles
['Boston Celtics' 'Los Angeles Lakers' 'Philadelphia 76ers'
 'Seattle SuperSonics' 'Phoenix Suns*' 'Atlanta Hawks*' 'Milwaukee Bucks*'
 'Kansas City Kings*' 'Houston Rockets*' 'San Antonio Spurs*'
 'Washington Bullets*' 'New York Knicks' 'Portland Trail Blazers*'
 'Cleveland Cavaliers' 'Indiana Pacers' 'San Diego Clippers'
 'New Jersey Nets' 'Chicago Bulls' 'Denver Nuggets'
 'Golden State Warriors' 'Utah Jazz' 'Detroit Pistons' 'Boston Celtics*'
 'Philadelphia 76ers*' 'Los Angeles Lakers*' 'New York Knicks*'
 'Chicago Bulls*' 'Indiana Pacers*' 'Washington Bullets' 'Atlanta Hawks'
 'Dallas Mavericks' 'Seattle SuperSonics*' 'Denver Nuggets*'
 'New Jersey Nets*' 'Portland Trail Blazers' 'Kansas City Kings'
 'Houston Rockets' 'Detroit Pistons*' 'Utah Jazz*' 'Dallas Mavericks*'
 'San Antonio Spurs' 'Cleveland Cavaliers*' 'Los Angeles Clippers'
 'Sacramento Kings*' 'Phoenix Suns' 'Golden State Warriors*'
 'Sacramento Kings' 'Charlotte Hornet

# Séparation en subsets

In [36]:
subset_sup, subset_inf = subsets(df, critère)

Si on veut travailler sur un seul subset par exemple, on le défini.

In [37]:
subset = subset_sup

# Préprocessing

In [38]:
X, y, scaler = preprocessing(subset, cible)

# Modélisation

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
trainer = ModelTrainer()

# Test des performances des modèles

In [40]:
best_model, scores = trainer.train_and_evaluate(X_train, y_train, X_test, y_test)

Model: RandomForest, Best Score: 0.6096, Best Params: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
Model: GradientBoosting, Best Score: 0.5975, Best Params: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Model: LogisticRegression, Best Score: 0.6019, Best Params: {'C': 0.1, 'penalty': 'l2'}
Model: SVC, Best Score: 0.6118, Best Params: {'C': 0.1, 'kernel': 'linear'}


In [ ]:
model_testing(best_model,X_test,y_test)

In [48]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred=best_model.predict(X_test)

scores = {    "accuracy": accuracy_score(y_test, y_pred),
              "classification_report": classification_report(y_test, y_pred, output_dict=True), 
              "confusion_matrix": confusion_matrix(y_test, y_pred)
              }
scores

{'accuracy': 0.576,
 'classification_report': {'CHAMPS': {'precision': 1.0,
   'recall': 0.5,
   'f1-score': 0.6666666666666666,
   'support': 2.0},
  'ECF': {'precision': 0.3333333333333333,
   'recall': 0.25,
   'f1-score': 0.2857142857142857,
   'support': 4.0},
  'ECFR': {'precision': 0.45454545454545453,
   'recall': 0.5263157894736842,
   'f1-score': 0.4878048780487805,
   'support': 19.0},
  'ECSF': {'precision': 0.18181818181818182,
   'recall': 0.2,
   'f1-score': 0.19047619047619047,
   'support': 10.0},
  'FINALS': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5.0},
  'NP': {'precision': 0.8305084745762712,
   'recall': 0.9245283018867925,
   'f1-score': 0.875,
   'support': 53.0},
  'WCF': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5.0},
  'WCFR': {'precision': 0.4444444444444444,
   'recall': 0.5,
   'f1-score': 0.47058823529411764,
   'support': 16.0},
  'WCSF': {'precision': 0.16666666666666666,
   'recall': 0.09090909090909091,
   'f1-

# Sauvegarder dans mlflow

In [70]:
sauvegarder_modele(best_model, "sklearn_models", experience_name, subset)

2024/12/20 23:25:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle sauvegardé avec MLflow dans l'expérience Classification_Playoffs, subset :       Rk                    Team   Age    MOV   SOS    SRS   ORtg   DRtg  \
0      1          Boston Celtics  27.3   7.79 -0.42   7.37  109.4  101.9   
1      2      Los Angeles Lakers  26.2   5.90 -0.51   5.40  109.5  103.9   
2      3      Philadelphia 76ers  27.0   4.22 -0.18   4.04  105.0  101.0   
3      4     Seattle SuperSonics  27.0   4.66 -0.42   4.24  105.8  101.2   
4      5            Phoenix Suns  26.5   3.60 -0.35   3.25  105.6  102.2   
...   ..                     ...   ...    ...   ...    ...    ...    ...   
1249  26       San Antonio Spurs  23.0  -6.49  0.69  -5.80  110.0  116.4   
1250  27  Portland Trail Blazers  24.1  -9.02  0.74  -8.29  108.3  117.5   
1251  28       Charlotte Hornets  25.1 -10.24  0.13 -10.12  109.3  119.8   
1252  29      Washington Wizards  24.9  -9.29  0.00  -9.29  110.5  119.6   
1253  30         Detroit Pistons  23.8  -9.11  0.05  -9.06  109.7  118.8   

     